# Iris Flower Classification with Scikit-Learn

![Iris](https://github.com/featurestoreorg/serverless-ml-course/raw/main/src/01-module/assets/iris.png)


In this notebook we will, 

1. Synthetic data generation for Iris Flowers
2. Split trainind data into train and test sets (one set each for features and labels)
3. Encode the label
4. Train a KNN Model using SkLearn
5. Evaluate model performance on the test set

In [ ]:
import random


def generate_flower(name, sepal_len_max, sepal_len_min, sepal_width_max, sepal_width_min, 
                    petal_len_max, petal_len_min, petal_width_max, petal_width_min):
    columns = ["sepal_length", "sepal_width", "petal_length", "petal_width"]
    flower = []
    flower.append(random.uniform(sepal_len_max, sepal_len_min))
    flower.append(random.uniform(sepal_width_max, sepal_width_min))
    flower.append(random.uniform(petal_len_max, petal_len_min))
    flower.append(random.uniform(petal_width_max, petal_width_min))
    df = pd.DataFrame(flower,columns)
    df['variety'] = name
    return flower


virginica_df = generate_flower("virginica" 6, 4.5, 3.8, 2.2, 7, 4.5, )
versicolor_df = generate_flower("versicolor", 8, 5.5, 3.5, 2.5, 2, 1, )
setosa_df =  generate_flower("setosa", 7, 5, 4.5, 2.5, 5, 3.5, )

# randomly pick one of these 3 and write it to the featurestore

We can see that our 3 different classes of iris flowers have different *petal_lengths* 
(although there are some overlapping regions between Versicolor and the two other varieties (Setoas, Virginica))

Split the DataFrame into 2: one DataFrame containing the *features* and one containing the *labels*.

In [ ]:
features = iris_df[["sepal_length", "sepal_width", "petal_length", "petal_width"]]
labels = iris_df[["variety"]]
features

In [ ]:
labels

We can split our features and labels into a **train_set** and a **test_set**. You split your data into a train_set and a test_set, because you want to train your model on only the train_set, and then evaluate its performance on data that was not seen during training, the test_set. This technique helps evaluate the ability of your model to accurately predict on data it has not seen before.

This looks as follows:

* **X_** is a vector of features, so **X_train** is a vector of features from the **train_set**. 
* **y_** is a scale of labels, so **y_train** is a scalar of labels from the **train_set**. 

Note: a vector is an array of values and a scalar is a single value.

Note: that mathematical convention is that a vector is denoted by an uppercase letter (hence "X") and a scalar is denoted by a lowercase letter (hence "y").

**X_test** is the features and **y_test** is the labels from our holdout **test_set**. The **test_set** is used to evaluate model performance after the model has been trained.


In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(features, labels, test_size=0.2)

Now we will do some **Feature Engineering**. 

We will transform the label from a categorical variable (a string) into a numerical variable (an int). Many machine learning training algorithms only take numerical values as inputs for training (and inference).

We can see that our original lables (**y_train** and **y_test**) are categorical variables. We will use Scikit-Learn's **LabelEncoder** to transform the strings into numbers.

In [ ]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
y_train_encoded=le.fit_transform(y_train['variety'])
y_test_encoded = le.transform(y_test['variety'])
y_test.head(8)

We can see that **y_test_encoded** has been transformed into a numerical variable (an int). **y_train_encoded** has been similarly transformed.

In [ ]:
y_test_encoded[0:8]

Now, we can fit a model to our features and labels from our training set (**X_train** and **y_train_encoded**). Fitting a model to a dataset is more commonly called "training a model".

In [ ]:
model = KNeighborsClassifier(n_neighbors=4)
model.fit(X_train, y_train_encoded) 

Now, we have trained our model. We can evaluate our model on the **test_set** to estimate its performance.
Notice that the model was trained to output the encoded labels (numbers).

In [ ]:
y_pred_encoded = model.predict(X_test)
y_pred_encoded

We can look at the predicted flower names by inverse transforming our numerical predictions back into their original string form. To perform the inverse transform, we need the **le** (LabelEncoder) object used to perform the orginal categorical to numerical mapping.

In [ ]:
le.inverse_transform(y_pred_encoded)

We can report on how accurate these predictions (**y_pred_encoded**) are compared to the labels (the actual results - **y_test_encoded**). 

In [ ]:
from sklearn.metrics import classification_report

metrics = classification_report(y_test_encoded, y_pred_encoded)
print(metrics)

In [ ]:
from sklearn.metrics import confusion_matrix

results = confusion_matrix(y_test_encoded, y_pred_encoded)
print(results)

In [ ]:
from matplotlib import pyplot

# Setosa = 0, Versicolor = 1, Virginica = 2

df_cm = pd.DataFrame(results, ['True Setosa', 'True Versicolor', 'True Virginica'],
                     ['Pred Setosa', 'Pred Versicolor', 'Pred Virginica'])

sns.heatmap(df_cm, annot=True)

Homework task

Rewrite the last two cells, but instead of computing the *classification_report* and the *confusion_matrix* with the encoded labels, use the unencoded labels. 

Are the results the same? Why?

In [ ]:
from sklearn.metrics import confusion_matrix

predictions_untransformed = le.inverse_transform(y_pred_encoded)
results = confusion_matrix(y_test, predictions_untransformed)
print(results)

In [ ]:
!pip install gradio --quiet
!pip install typing-extensions==4.3.0

In [ ]:
import gradio as gr
import numpy as np


def iris(sl, sw, pl, pw):
    input_list = []
    input_list.append(sl)
    input_list.append(sw)
    input_list.append(pl)
    input_list.append(pw)
    res = model.predict(np.asarray(input_list).reshape(1, -1)) 
    # Convert the numerical representation of the label back to it's original iris flower name.
    # le.inverse_transform returns a list of flower names with only 1 entry, so we add '[0]' to 
    # the list returned by le.inverse_transform(..) to return only the iris flower name (not the list)
    return le.inverse_transform(res)[0]

demo = gr.Interface(
    fn=iris,
    title="Iris Flower Predictive Analytics",
    description="Experiment with sepal/petal lengths/widths to predict which flower it is.",
    allow_flagging="never",
    inputs=[
        gr.inputs.Number(default=1.0, label="sepal length (cm)"),
        gr.inputs.Number(default=1.0, label="sepal width (cm)"),
        gr.inputs.Number(default=1.0, label="petal length (cm)"),
        gr.inputs.Number(default=1.0, label="petal width (cm)"),
        ],
    outputs="text")

demo.launch(share=True)